In [1]:
import os
runningFromColab = False
if 'CGROUP_MEMORY_EVENTS' in os.environ and 'colab' in os.environ['CGROUP_MEMORY_EVENTS']:
  runningFromColab = True

In [2]:
if runningFromColab:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
if runningFromColab:
  %cd /content/drive/MyDrive/ColabProjects

/content/drive/MyDrive/ColabProjects


In [13]:
if runningFromColab:
  !git clone https://github.com/OdyAsh/nlp-image-captioning.git

fatal: destination path 'nlp-image-captioning' already exists and is not an empty directory.


In [4]:
if runningFromColab:
  %cd /content/drive/MyDrive/ColabProjects/nlp-image-captioning

/content/drive/MyDrive/ColabProjects/nlp-image-captioning


In [14]:
if runningFromColab:
  !git pull

Already up to date.


In [10]:
if runningFromColab:
  try:
    import condacolab
    condacolab.install()
  except:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# now restart the kernel

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:26
🔁 Restarting kernel...


In [13]:
if runningFromColab:
  !conda env create -f environment.yml
  # !conda update conda -y -q
  # !source /usr/local/etc/profile.d/conda.sh
  # !conda init 
  # !conda install -n root _license -y -q
  # !source activate myenv


CondaValueError: could not parse 'name: .conda' in: environment.yml



In [ ]:
if runningFromColab:
  import sys
  sys.path.insert(0, '/usr/local/bin/conda')